# Functional Connectivity and Network Analysis for BeveL Betaseries 

Inputs: betaseries files for BeveL participants (n=85) drawn from 4 conditions: choice, reward taste, punishment taste, neutral rinse

Analysis workflow is mapped off this paper: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5429248/

In [29]:
import glob
import os
import networkx as nx
import numpy as np
import pandas as pd
import bz2
import pickle
import community
import pdb
import statistics
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt

from visbrain.objects import ConnectObj, SceneObj, SourceObj, BrainObj
from visbrain.io import download_file

## Load in the data

### Find the path to the data

In [2]:
#Find the path to data
file_list = glob.glob('/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/sync_9_16/reward/*.txt')

In [3]:
#Check the files found
print(file_list)

['/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/sync_9_16/reward/sub-001_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/sync_9_16/reward/sub-002_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/sync_9_16/reward/sub-003_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/sync_9_16/reward/sub-004_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/sync_9_16/reward/sub-006_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/sync_9_16/reward/sub-007_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/sync_9_16/reward/sub-009_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/sync_9_16/reward/sub-010_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/sync_9_16/reward/sub-012_reward.txt', '/Users/jennygilbert/Documents/betas

In [4]:
#Check to see how many participants 
len(file_list)

41

### Make a dictionary to read in the files to pandas

In [5]:
my_dict={}
for item in file_list:
    name=item.split('/')[8].split('.')[0]
    print(name)
    my_dict.setdefault(name, []).append(item)

sub-001_reward
sub-002_reward
sub-003_reward
sub-004_reward
sub-006_reward
sub-007_reward
sub-009_reward
sub-010_reward
sub-012_reward
sub-014_reward
sub-016_reward
sub-017_reward
sub-018_reward
sub-019_reward
sub-020_reward
sub-021_reward
sub-022_reward
sub-024_reward
sub-025_reward
sub-026_reward
sub-027_reward
sub-028_reward
sub-029_reward
sub-031_reward
sub-032_reward
sub-034_reward
sub-037_reward
sub-038_reward
sub-040_reward
sub-042_reward
sub-043_reward
sub-045_reward
sub-046_reward
sub-048_reward
sub-054_reward
sub-056_reward
sub-058_reward
sub-060_reward
sub-061_reward
sub-063_reward
sub-064_reward


In [6]:
# Checking to make sure its populated
my_dict['sub-024_reward']

['/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/sync_9_16/reward/sub-024_reward.txt']

### Read the data from the dictionary into pandas

In [7]:
data_dict={}
for key, value in my_dict.items():
    for i in value:
        data_dict.setdefault(key, []).append(pd.read_csv(i, sep='\t' ,header=None,index_col=False))
#         data_dict[key]= pd.read_csv(i, header=None,index_col=False)

In [8]:
#Check the dictionary
data_dict.keys()

dict_keys(['sub-001_reward', 'sub-002_reward', 'sub-003_reward', 'sub-004_reward', 'sub-006_reward', 'sub-007_reward', 'sub-009_reward', 'sub-010_reward', 'sub-012_reward', 'sub-014_reward', 'sub-016_reward', 'sub-017_reward', 'sub-018_reward', 'sub-019_reward', 'sub-020_reward', 'sub-021_reward', 'sub-022_reward', 'sub-024_reward', 'sub-025_reward', 'sub-026_reward', 'sub-027_reward', 'sub-028_reward', 'sub-029_reward', 'sub-031_reward', 'sub-032_reward', 'sub-034_reward', 'sub-037_reward', 'sub-038_reward', 'sub-040_reward', 'sub-042_reward', 'sub-043_reward', 'sub-045_reward', 'sub-046_reward', 'sub-048_reward', 'sub-054_reward', 'sub-056_reward', 'sub-058_reward', 'sub-060_reward', 'sub-061_reward', 'sub-063_reward', 'sub-064_reward'])

In [9]:
#Check for the dataframe
data_dict['sub-058_reward']

[            0           1           2           3           4           5   \
 0    68.360291   45.647396  -16.965845   84.948692   39.198357  -30.621855   
 1   102.598129   17.282980  -26.879036  -86.759956   39.177406   13.087736   
 2    76.307739   29.793316   10.971916   58.552471   -0.441348   -0.808222   
 3   210.930359  159.814880   45.043007  133.201599  104.217514  -29.165623   
 4   -91.944801   84.479836    4.762266    9.267733   37.287460   23.768517   
 5   115.271950   48.284798  -24.895542   40.839394   35.163708   -9.680651   
 6    -6.440373   66.152054  -62.975662   13.151779    1.603159    8.119939   
 7   -97.147217  279.945587  -36.686951  -53.525486   39.616070  -19.276733   
 8   178.807526  186.360535   36.817814   50.739586   18.883062   40.366669   
 9  -224.359589 -178.982056    3.218035  -28.598566  -64.436050  -36.720451   
 10 -121.210144   12.318864   46.015888   54.661221  -61.690392  -65.814743   
 11  -59.784725  -70.944595  -53.706959  -40.182983 

### Append the dictionary with correlation matrix

In [13]:
for key, value in data_dict.items():
    value[0]
    #pdb.set_trace()
    cor_matrix = value[0].corr()
    data_dict[key].append(cor_matrix)
    

### Reminder
The first element in the value of data_dict is the raw data, the second element is the correlation matrix

In [16]:
list(data_dict.values())[0][1]

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,1.000000,-0.387656,0.211592,0.164064,0.144819,0.019931,0.309698,0.337296,-0.241508,-0.161034,...,0.334306,0.097368,0.165362,0.094875,-0.123572,0.081814,-0.149071,-0.214182,-0.168050,0.395889
1,-0.387656,1.000000,0.165126,0.255205,0.093344,-0.146389,0.098567,-0.017510,0.431533,0.511078,...,0.119476,0.341087,-0.059887,0.308616,0.308986,0.148851,0.228967,0.119264,0.246379,-0.032476
2,0.211592,0.165126,1.000000,0.590416,0.196795,0.184780,0.094666,0.371306,0.338171,0.386672,...,0.561977,0.204279,0.218680,0.362618,0.312578,0.462059,0.227362,0.247267,0.448720,0.362352
3,0.164064,0.255205,0.590416,1.000000,0.368647,0.413830,0.307527,0.504505,0.428758,0.525461,...,0.568746,0.357345,0.224775,0.542676,0.447326,0.269727,0.406708,0.392709,0.672724,0.295437
4,0.144819,0.093344,0.196795,0.368647,1.000000,0.245400,0.346915,0.339502,0.176416,0.330315,...,0.385531,0.204433,-0.053393,0.165440,0.178603,-0.096474,0.296524,0.232314,0.223810,0.100650
5,0.019931,-0.146389,0.184780,0.413830,0.245400,1.000000,-0.019068,0.092953,0.234423,0.318239,...,0.268605,0.026990,0.232833,0.343155,0.161930,0.252086,0.343304,0.246862,0.412001,-0.097353
6,0.309698,0.098567,0.094666,0.307527,0.346915,-0.019068,1.000000,0.520580,-0.094823,0.207187,...,0.077141,0.265915,0.125971,0.126474,0.016132,-0.034395,-0.074804,-0.040530,0.167133,0.120287
7,0.337296,-0.017510,0.371306,0.504505,0.339502,0.092953,0.520580,1.000000,0.040149,0.336363,...,0.213149,0.260743,0.054938,0.367376,0.140925,0.059785,0.148818,0.253072,0.335878,0.352843
8,-0.241508,0.431533,0.338171,0.428758,0.176416,0.234423,-0.094823,0.040149,1.000000,0.542766,...,0.292772,0.167705,0.157160,0.310037,0.771901,0.319895,0.616318,0.610181,0.418539,-0.120059
9,-0.161034,0.511078,0.386672,0.525461,0.330315,0.318239,0.207187,0.336363,0.542766,1.000000,...,0.247014,0.331838,0.039666,0.462238,0.579128,0.206088,0.464545,0.417281,0.430909,-0.018946


### Function to create a graph with positive or negative values and minimum correlation value

In [17]:
def create_corr_network_5(G, corr_direction, min_correlation):

    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
    return(H)

### Function to make a graph object BY SUBJECT
This will return:
- The edges (noramlized R correlation matrix, in pandas dataframe)
- The correlations (absolute value of the edges in a numpy dataframe)
- The mean_FC (the mean functional connectivity per subject/run)
- The graphs (this will contain the raw graph object G as well as the the partion values from the modularity calculation)

In [43]:
def make_graphs(list_o_data, direction, min_cor):
    edge_dict={}
    cor_dict={}
    FC_dict={}
    graph_dict={}
    for key, val_list in list_o_data.items():
        for i in val_list[1]:
            #i=i.set_index(labels.ID)
            #i.rename(columns=labels.ID, inplace=True)
            ########################################
            edge_dict.setdefault(key, []).append(i)
            ########################################
            cor_matrix = np.asmatrix(i)
            x=abs(cor_matrix)
            mu=x.mean()
            ########################################
            cor_dict.setdefault(key, []).append(x)
            ########################################
            FC_dict.setdefault(key, []).append(mu)
            ########################################
            G = nx.from_numpy_matrix(cor_matrix)
            #for i, nlrow in labels.iterrows():
                #G.node[i].update(nlrow[0:].to_dict())
            ########################################
            graph_dict.setdefault(key, []).append(G)
            ########################################
            #partition = community.best_partition(create_corr_network_5(G, direction,min_cor))
            ########################################
            #graph_dict.setdefault(key, []).append(partition)
            ########################################
    return({'edges':edge_dict, 'correlations':cor_dict, 'mean_FC':FC_dict, 'graphs':graph_dict})

In [44]:
GRAPHS = make_graphs(data_dict, "positive", 0)

AttributeError: 'int' object has no attribute 'mean'

In [34]:
GRAPHS.keys()

dict_keys(['edges', 'correlations', 'mean_FC', 'graphs'])

In [45]:
#Something is wrong here
GRAPHS['correlations']['sub-001_reward']

[matrix([[0]]),
 matrix([[1]]),
 matrix([[2]]),
 matrix([[3]]),
 matrix([[4]]),
 matrix([[5]]),
 matrix([[6]]),
 matrix([[7]]),
 matrix([[8]]),
 matrix([[9]]),
 matrix([[10]]),
 matrix([[11]]),
 matrix([[12]]),
 matrix([[13]]),
 matrix([[14]]),
 matrix([[15]]),
 matrix([[16]]),
 matrix([[17]]),
 matrix([[18]]),
 matrix([[19]]),
 matrix([[20]]),
 matrix([[21]]),
 matrix([[22]]),
 matrix([[23]]),
 matrix([[24]]),
 matrix([[25]]),
 matrix([[26]])]

In [38]:
#Creates a dictionary with node names
mapping={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
         9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_R", 13: "mOFC_L", 14:"mOFC_R", 15:"Oral_somatosensory_cortex_L", 16:"Oral_somatosensory_cortex_R", 17:"Precuneus_L", 
         18:"Precuneus_R", 19:"Ventral_striatum_L", 20:"Ventral_striatum_R", 21:"vlPFC_L", 22:"vlPFC_R", 23:"vlThalamus_L" , 24:"vlThalamus_R", 25: "vmPFC_L", 26: "vmPFC_R"}

#relabels the nodes to match the ROI names
G = nx.relabel_nodes(GRAPHS,mapping)

AttributeError: 'dict' object has no attribute 'add_nodes_from'